<a href="https://colab.research.google.com/github/Momilijaz96/VGG16_CIFAR100_Optim/blob/main/VGG16_CIFAR100(SGD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import models,transforms,datasets


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Import CIFAR-100 and Resize

In [17]:
# train_transform = transforms.Compose([transforms.Resize(64),
#                                    transforms.RandomHorizontalFlip(),
#                                    transforms.RandomRotation(15),
#                                    transforms.ToTensor(),
#                                    transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])
train_transform =  transforms.Compose([
            #transforms.ToPILImage(),
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))
        ])
test_transform = transforms.Compose([transforms.Resize(32),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])

In [18]:
train_set = datasets.CIFAR100(root = '~/data/pytorch_cifar100', train = True, download = False, transform = train_transform)

In [19]:
test_set = datasets.CIFAR100(root = '~/data/pytorch_cifar100', train = False, download = True, transform = test_transform)

Files already downloaded and verified


In [7]:
train_set[0][0].shape

torch.Size([3, 64, 64])

In [8]:
test_set[0][0].shape

torch.Size([3, 64, 64])

# VGG16

In [20]:
# Number of classes in the dataset
num_classes = 100

# Batch size for training
batch_size = 128

# Number of epochs
num_epochs = 20

In [21]:
vgg16_model = models.vgg16(pretrained = True) #130million+ parameters

#set_parameter_requires_grad for classifier layers
'''
for param in vgg16_model.parameters():
  param.requires_grad=False
'''

#Modify last layer out classes
vgg16_model.classifier[6].out_features = num_classes

vgg16_model = vgg16_model.to(device)

In [12]:
print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [22]:
criterion = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(vgg16_model.parameters(), lr = 0.001, momentum = 0.9)
lr_scheduler = optim.lr_scheduler.StepLR(sgd_optimizer, step_size = 10, gamma = 0.1)

train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_set, batch_size = batch_size, shuffle = True)

In [23]:
def fit(model, dataset, optimizer, scheduler, criterion):
  #Set model to training mode
  model.train()
  scheduler.step()
  #Iterate over data
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    #Reset the gradients
    optimizer.zero_grad()
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

In [24]:
def validate(model, dataset, criterion):
  #set model to evaluation mode
  model.eval()
  acc = 0
  loss = []
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    out = model(data)
    #Get loss
    l = criterion(out, targets)
    loss.append(l.item())
    #Get index of class label
    _,preds = torch.max(out.data,1)
    #Get accuracy
    acc += torch.sum(preds == targets).item()

  return 100*acc/len(dataset.dataset), np.mean(np.array(loss))

In [25]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []

#Loop for num_epochs
for epoch in range(num_epochs):
  #Fit model
  fit(vgg16_model, train_dataloader, sgd_optimizer, lr_scheduler, criterion)

  #Train validation
  trn_acc, trn_lss = validate(vgg16_model, train_dataloader, criterion)
  train_loss.append(trn_lss)
  train_acc.append(trn_acc)

  #Test validation
  tst_acc, tst_lss = validate(vgg16_model, test_dataloader, criterion)
  test_loss.append(tst_lss)
  test_acc.append(tst_acc)

  print(f'Epoch:{epoch+1}\t Train_Loss:{train_loss[-1]:.4f}\t Train_Acc:{train_acc[-1]:.4f}\t Test_Loss:{test_loss[-1]:.4f}\t Test_Acc:{test_acc[-1]:.4f}')


Epoch:1	 Train_Loss:3.9068	 Train_Acc:10.4700	 Test_Loss:3.9562	 Test_Acc:9.8500
Epoch:2	 Train_Loss:2.9805	 Train_Acc:24.2420	 Test_Loss:2.9220	 Test_Acc:24.7600
Epoch:3	 Train_Loss:2.5119	 Train_Acc:33.4680	 Test_Loss:2.4204	 Test_Acc:34.6300
Epoch:4	 Train_Loss:2.2395	 Train_Acc:40.0080	 Test_Loss:2.1346	 Test_Acc:40.8900
Epoch:5	 Train_Loss:2.0590	 Train_Acc:44.1460	 Test_Loss:1.9786	 Test_Acc:45.5300
Epoch:6	 Train_Loss:1.8844	 Train_Acc:47.9700	 Test_Loss:1.8371	 Test_Acc:48.7000
Epoch:7	 Train_Loss:1.8168	 Train_Acc:49.8300	 Test_Loss:1.7652	 Test_Acc:50.4600
Epoch:8	 Train_Loss:1.7607	 Train_Acc:50.5460	 Test_Loss:1.7059	 Test_Acc:51.3400
Epoch:9	 Train_Loss:1.6302	 Train_Acc:54.0840	 Test_Loss:1.6458	 Test_Acc:53.3300
Epoch:10	 Train_Loss:1.5183	 Train_Acc:56.9260	 Test_Loss:1.5430	 Test_Acc:56.2500
Epoch:11	 Train_Loss:1.4962	 Train_Acc:57.4800	 Test_Loss:1.5229	 Test_Acc:56.7200
Epoch:12	 Train_Loss:1.4829	 Train_Acc:57.6540	 Test_Loss:1.5083	 Test_Acc:56.8800
Epoch:13	 Trai

# Save Model

In [ ]:
torch.save(vgg16_model.state_dict(), 'VGG16(SGD)')

time: 1.96 s (started: 2021-09-04 06:02:29 +00:00)
